In [3]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

NotFoundError: dlopen(/opt/anaconda3/lib/python3.8/site-packages/tensorflow_quantum/core/ops/_tfq_simulate_ops.so, 6): Symbol not found: __ZN10tensorflow8OpKernel11TraceStringEPNS_15OpKernelContextEb
  Referenced from: /opt/anaconda3/lib/python3.8/site-packages/tensorflow_quantum/core/ops/_tfq_simulate_ops.so
  Expected in: /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/../libtensorflow_framework.2.dylib
 in /opt/anaconda3/lib/python3.8/site-packages/tensorflow_quantum/core/ops/_tfq_simulate_ops.so

In [ ]:
# variables
symbols = sympy.symbols('a b c d')

q0, q1 = cirq.LineQubit.range(2)


# create variational circuit
def get_var_form(symbols):
  circuit = cirq.Circuit(
    cirq.rx(symbols[0]).on(q0),
    cirq.rx(symbols[1]).on(q1), 
    cirq.ry(symbols[2]).on(q0),
    cirq.ry(symbols[3]).on(q1),
    # Monte Carlo Simulation of Noise
    cirq.bit_flip(p=0.02)(q0),
    cirq.bit_flip(p=0.02)(q1), 
    cirq.CNOT(control=q0, target=q1),
    )
  return circuit


# Circuit Visualization
circuit = get_var_form(symbols)
SVGCircuit(circuit)

simulator = cirq.Simulator()
#initialize parameters to all 0's
params = [0] * 4
params = tf.convert_to_tensor(
    params, dtype=tf.float32, dtype_hint=None, name=None
)
print('starting parameters:')
print(params)
param0, param1, param2, param3 = 0, 0, 0, 0 

#  measurements
reps = 1000

# loss function
def loss(params):
  qc = get_var_form(symbols)
  qc.append(cirq.measure(q0, q1, key='result'))
  resolver = cirq.ParamResolver({symbols[0]: params[0], symbols[1]: params[1], symbols[2]: params[2], symbols[3]: params[3]})
  results = simulator.run(qc, resolver, repetitions=reps)
  counter = results.histogram(key='result')

  # loss as euclidean distance to 
    
    
    #(1/2, 1/2)
    
    
    
  loss = (counter[1]/reps - 0.5)**2 + (counter[2]/reps - 0.5)**2
  return loss
l = loss(params)
print('starting loss:' + str(l))

# optimize
from scipy.optimize import minimize
from scipy import optimize

# optimize 2
res = minimize(loss, params, method='COBYLA', tol=0.001) ## COBYLA # powell # Nelder-Mead # BFGS
print(res)

#VER
resolver = cirq.ParamResolver({symbols[0]: res.x[0], symbols[1]: res.x[1], symbols[2]: res.x[2], symbols[3]: res.x[3]})
qc = get_var_form(symbols)
qc.append(cirq.measure(q0, q1, key='result'))
results = simulator.run(qc, resolver, repetitions=reps)
counter = results.histogram(key='result')
loss = (counter[1]/reps - 0.5)**2 + (counter[2]/reps - 0.5)**2
print('sampling result: ')
print(counter)
print('loss: ' + str(loss))

# STATE
qc0 = get_var_form(symbols)
state = simulator.simulate(qc0, resolver).final_state
print('state vector: ')
print(state)